In [1]:
import numpy as np
import tensorflow as tf
import os 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

from numpy import array
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Embedding, Dense

2021-11-18 10:43:24.918502: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [12]:
reviews = ['muy mala', 'mala comida', 'no volveria', 'no atienden bien', 'pesima atencion',
          'muy buena', 'comida muy rica', 'lo recomiendo', 'buena atencion', 'lindo lugar']

labels = np.array([1,1,1,1,1,0,0,0,0,0])

In [13]:
Vocab_size = 50
encoded_reviews = [one_hot(d, Vocab_size) for d in reviews] #convertir cada palabra en su indice one_hot
print(f'encoded reviews: {encoded_reviews}')

encoded reviews: [[6, 43], [43, 24], [19, 35], [19, 15, 31], [8, 41], [6, 3], [24, 6, 41], [35, 26], [3, 41], [28, 12]]


El largo de cada review codificada es igual a la cantidad de palabras que contiene.
Luego se aplica un padeo para que todas las reviews tengan la misma cantidad de palabras. 

In [14]:
max_length = 4 
padded_reviews = pad_sequences(encoded_reviews, maxlen=max_length, padding='post')
print(padded_reviews)

[[ 6 43  0  0]
 [43 24  0  0]
 [19 35  0  0]
 [19 15 31  0]
 [ 8 41  0  0]
 [ 6  3  0  0]
 [24  6 41  0]
 [35 26  0  0]
 [ 3 41  0  0]
 [28 12  0  0]]


In [15]:
model = Sequential([
    Embedding(input_dim=Vocab_size, output_dim=8, input_length=max_length),
    Flatten(),
    Dense(1, activation='sigmoid')
    ])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 4, 8)              400       
_________________________________________________________________
flatten_3 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(padded_reviews, labels, epochs=100)

2021-11-18 11:29:03.981115: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-18 11:29:03.998337: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 3199980000 Hz


Epoch 1/100


2021-11-18 11:29:04.293782: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11


1/1 [==============================] - 1s 873ms/step - loss: 0.6900 - acc: 0.6000
Epoch 2/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6887 - acc: 0.7000
Epoch 3/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6875 - acc: 0.7000
Epoch 4/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6863 - acc: 0.8000
Epoch 5/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6851 - acc: 0.8000
Epoch 6/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6839 - acc: 0.8000
Epoch 7/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6827 - acc: 0.8000
Epoch 8/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6815 - acc: 0.8000
Epoch 9/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6803 - acc: 0.8000
Epoch 10/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6791 - acc: 0.8000
Epoch 11/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6779 - acc: 0.8

2021-11-18 11:29:04.868132: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11


Epoch 57/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6128 - acc: 1.0000
Epoch 58/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6111 - acc: 1.0000
Epoch 59/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6093 - acc: 1.0000
Epoch 60/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6076 - acc: 1.0000
Epoch 61/100
1/1 [==============================] - 0s 3ms/step - loss: 0.6058 - acc: 1.0000
Epoch 62/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6041 - acc: 1.0000
Epoch 63/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6023 - acc: 1.0000
Epoch 64/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6005 - acc: 1.0000
Epoch 65/100
1/1 [==============================] - 0s 3ms/step - loss: 0.5987 - acc: 1.0000
Epoch 66/100
1/1 [==============================] - 0s 2ms/step - loss: 0.5969 - acc: 1.0000
Epoch 67/100
1/1 [==============================] - 0s 2ms/step - loss

Analizo la capa de embeddings

In [20]:
model.layers[0].get_weights()[0].shape

(50, 8)

De esta forma se construye esta look-up table en donde se definen representaciones vectoriales complejas de cada palabra, para luego usar esas representaciones en el entrenamiento de modelos. 
El embedding entonces es util para reducir y controlar la dimensionalidad de la entrada. Además, sirve también para el entrenamiento, ya que palabras similares tendran vectores similares (si medimos la distancia entre los vectores de cada palabra) por lo que se dice que logra comprender el contexto de las palabras. 

In [22]:
model.layers[0].get_weights()[0]

array([[-0.03654153,  0.00487402,  0.02915067,  0.01064066, -0.02204375,
         0.00550731,  0.01894402, -0.008373  ],
       [ 0.01398249,  0.04996587,  0.03504917, -0.04931144, -0.00684848,
        -0.01316261,  0.0144709 ,  0.01086054],
       [ 0.01340405,  0.02136347, -0.04304823,  0.00463307, -0.03633474,
        -0.01735854,  0.00670094, -0.04261947],
       [ 0.06213295,  0.13782373,  0.1406892 , -0.12761796,  0.06027074,
         0.16188532,  0.10159532,  0.06858815],
       [ 0.03233233, -0.01768253,  0.03233585, -0.03782239, -0.01496118,
         0.03388165,  0.00948422,  0.01655096],
       [ 0.03591115, -0.03086826,  0.02316078,  0.01327726,  0.00955842,
         0.02788012, -0.00577217,  0.04274372],
       [ 0.13293459, -0.11894173,  0.1089919 ,  0.0626514 ,  0.13174963,
         0.08455895, -0.13400292,  0.14563218],
       [-0.04397264,  0.04545008, -0.00553574, -0.0047638 , -0.0315155 ,
        -0.03816116,  0.0435478 , -0.03272069],
       [ 0.15327074, -0.15201841